In [1]:
from pymatreader import read_mat
import pandas as pd
import numpy as np
import scipy.io as sio

In [39]:
def standardise(array):
    standardise = lambda x, y, z: (x-y)/z
    csistd = np.frompyfunc(standardise, 3, 1)
    mean = np.mean(array)
    std = np.std(array)
    return csistd(array,mean,std)

def segmentation(df):
    #Window size is 256, stride is 128
    segmented = pd.DataFrame({'Range': 'string', 'Sample': []})
    stride = 128
    startIndex = 0
    lastIndex = 256
    while(lastIndex < df.shape[0]):
        # print('In loop {} {}'.format(lastIndex, df.shape[0]))
        sample = df['Standardised']
        sample = sample.iloc[startIndex:lastIndex].to_numpy()
        index = '{}-{}'.format(startIndex, lastIndex)
        newRow = pd.Series({'Range' : index, 'Sample' : sample})
        segmented = pd.concat([segmented, newRow.to_frame().T], ignore_index=True)
        startIndex += stride
        lastIndex += stride
    # label = df['label'].loc[lastIndex]
    # print(label)
    # segmented['Label'] = [label for sample in segmented['Range']]
    # print('Done segmenting')
    return segmented
    
def preprocess(df, active, steady):
    print('In preprocessing')
    # print('{} {}'.format(active,steady))
    preprocessed = pd.DataFrame({'Range': '', 'Sample': [], 'Label': ''})
    #Extracting the amplitude
    df['CSI'] = np.abs(df['CSI'])
    # print('Calculated amp')
    #Using the equation from CSITime, standardize each sample
    df['Standardised'] = [standardise(sample) for sample in df["CSI"]]
    df.drop("CSI", inplace = True, axis = 1)
    
    #Separate the sample to the active and steady state
    intF = df.loc[df["label"] == '{}'.format(active)]
    intS = df.loc[df["label"] == '{}'.format(steady)]
    # print(intS.head())
    # print('Grouped')
    segmentF = segmentation(intF)
    segmentF['Label'] = [active for sample in segmentF['Range']]
    segmentS = segmentation(intS)
    segmentS['Label'] = [steady for sample in segmentS['Range']]
    
    # print('Segmented')
    preprocessed = pd.concat([preprocessed, segmentF], ignore_index=True)
    preprocessed = pd.concat([preprocessed, segmentS], ignore_index=True)
    
    # print('Preprocessed')
    
    
    return preprocessed

In [40]:
data = {}
subjectPair = ['S1_S47', 'S2_S22']
final = pd.DataFrame({'Range': 'string', 'Sample': [], 'Label': ''}) 

for pair in subjectPair: #Iterate through each pair
    for intera in range(1,3): #Index of interaction
        for trial in range(1,3): #Index of trial
            # print(('C:\\Users\\Karlj\\dissertation\\wifimod\\{0}\\I{1}\\{0}_I{1}_T{2}.mat'.format(pair, intera, trial)))
            data = read_mat('C:\\Users\\Karlj\\dissertation\\wifimod\\{0}\\I{1}\\{0}_I{1}_T{2}.mat'.format(pair, intera, trial))
            df = pd.DataFrame(data['Raw_Cell_Matrix'])
            df.drop(['Nrx', 'Ntx', 'noise','agc','RSSI_a', 'RSSI_b','RSSI_c', "timestamp_low"], axis=1, inplace=True)
            active = 'I{}'.format(intera)
            preprocessed = preprocess(df, active, 'I13')
            final = pd.concat([final, preprocessed], ignore_index=True)
print(final.head())
            
            

In preprocessing
In preprocessing
In preprocessing
In preprocessing
In preprocessing
In preprocessing
In preprocessing
In preprocessing
     Range                                             Sample Label
0    0-256  [[[[-1.1555606530164928 -1.0370904823417137 -1...    I1
1  128-384  [[[[-0.7881556491245104 -0.38491921496522924 -...    I1
2  256-512  [[[[-1.0722477059962134 -0.9521059014768991 -1...    I1
3  384-640  [[[[-0.3467121707683465 -0.33774440045954907 -...    I1
4  512-768  [[[[-0.20748885033841907 0.01253283461941942 -...    I1


In [42]:
print(final['Label'].unique())

['I1' 'I13' 'I2']
